In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics

In [2]:
data_dir = Path("../input/home-data-for-ml-course/")
df_train = pd.read_csv(data_dir / "train.csv")
df_test = pd.read_csv(data_dir / "test.csv")

In [3]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# function for comparing different approaches
def score_dataset(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [6]:
X_full = df_train.copy()
X_test = df_test.copy()

### Explore Outliners

### Explore Sale Price

### Combine training data and testing data

In [7]:
X_full.shape

(1460, 81)

In [8]:
X_test.shape

(1459, 80)

In [9]:
ntrain = X_full.shape[0]
ntest = X_test.shape[0]
y_train = X_full.SalePrice.values
all_data = pd.concat((X_full, X_test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2919, 80)


In [10]:
all_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


### Check Data

In [11]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in all_data.columns if
                    all_data[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in all_data.columns if 
                all_data[cname].dtype in ['int64', 'float64']]

In [12]:
#Get unique values of all categorical_cols for validation
for col in categorical_cols:
    print(col)
    print(all_data[col].unique())

MSZoning
['RL' 'RM' 'C (all)' 'FV' 'RH' nan]
Street
['Pave' 'Grvl']
Alley
[nan 'Grvl' 'Pave']
LotShape
['Reg' 'IR1' 'IR2' 'IR3']
LandContour
['Lvl' 'Bnk' 'Low' 'HLS']
Utilities
['AllPub' 'NoSeWa' nan]
LotConfig
['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
LandSlope
['Gtl' 'Mod' 'Sev']
Neighborhood
['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1
['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2
['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
BldgType
['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle
['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle
['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
RoofMatl
['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
Exterior1st
['VinylSd' '

In [13]:
pd.set_option('display.max_rows', 1500)

#check if GarageYrBlt < YearBuilt
garage_year = all_data[all_data['GarageYrBlt'] < all_data['YearBuilt']]
garage_year[['GarageYrBlt','YearBuilt']]

,GarageYrBlt,YearBuilt
29,1920.0,1927
93,1900.0,1910
324,1961.0,1967
600,2003.0,2005
736,1949.0,1950
1103,1954.0,1959
1376,1925.0,1930
1414,1922.0,1923
1418,1962.0,1963
1521,1956.0,1959


In [14]:
#check if YearRemodAdd < YearBuilt
remod_year = all_data[all_data['YearRemodAdd'] < all_data['YearBuilt']]
remod_year[['YearRemodAdd','YearBuilt']]

,YearRemodAdd,YearBuilt
1876,2001,2002


In [15]:
#correct typo of exterior2nd_dict
exterior2nd_dict = {"CmentBd" : "CemntBd",
                    "Wd Shng" : "Wd Sdng",
                    "Brk Cmn" : "BrkComm"
                   }

all_data['Exterior2nd'] = all_data['Exterior2nd'].replace(exterior2nd_dict)

#correct GarageYrBlt when it is earlier than YearBuilt  
all_data["GarageYrBlt"] = all_data["GarageYrBlt"].where(all_data.GarageYrBlt < all_data.YearBuilt, all_data.YearBuilt) 

#correct YearRemodAdd when it is earlier than YearBuilt
all_data["YearRemodAdd"] = all_data["YearRemodAdd"].where(all_data.YearRemodAdd < all_data.YearBuilt, all_data.YearBuilt) 

### Check missing values and decide how to fill them

In [16]:
#check any columns with null values
all_cols_na = all_data.columns[all_data.isnull().any()]
all_cols_na

Index(['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType'],
      dtype='object')

In [17]:
all_data_na = (all_data.isnull().sum() / len(X_full)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio
PoolQC,199.246575
MiscFeature,192.739726
Alley,186.369863
Fence,160.821918
FireplaceQu,97.260274
LotFrontage,33.287671
GarageFinish,10.890411
GarageQual,10.890411
GarageCond,10.890411
GarageType,10.753425


In [18]:
#check if missing GarageYrBlt but there is actually a Garage
all_data[all_data['GarageYrBlt'].isnull() & all_data['GarageArea'] > 0]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


Since null GarageYrBlt means there is no garage, we'll fill it as 0. For the following columns, we'll fill them NA / None / 0 according to the data description. LotFrontage will be median of Neighbour's. For the rest, we'll fill with SimpleImputer.

'Alley' : "NA" 'MasVnrType': "None" 'MasVnrArea': 0 'BsmtQual': "NA"
'BsmtCond': "NA" 'BsmtExposure' : "NA" 'BsmtFinType1': "NA" 'BsmtFinType2': "NA" 'FireplaceQu': "NA" 'GarageType': "NA" 'GarageYrBlt': 0
'GarageFinish': "NA" 'GarageQual': "NA" 'GarageCond': "NA" 'PoolQC': "NA" 'Fence': "NA"
'MiscFeature': "NA"

In [19]:
#Fill null values with custom values

cols_with_NA = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu',
                'GarageType','GarageFinish','GarageQual','GarageCond',
               'BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtCond','BsmtQual','MasVnrType']
cols_with_zero = ['LotFrontage','GarageYrBlt','MasVnrArea','GarageCars','BsmtFullBath','BsmtHalfBath',
                  'BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea','BsmtFinSF1','BsmtFinSF2']
other_cols= list(set(all_cols_na) - set(cols_with_NA) - set(cols_with_zero))

other_cols

['Exterior2nd',
 'Functional',
 'Utilities',
 'Electrical',
 'KitchenQual',
 'MSZoning',
 'SaleType',
 'Exterior1st']

In [20]:
#for col in cols_with_NA: all_data[col].fillna("NA",inplace=True)
#for col in cols_with_zero: all_data[col].fillna(0, inplace=True)

In [21]:
cols_with_missing = [col for col in  all_data.columns
                      if all_data[col].isnull().any()]
                     

all_data = all_data.drop(cols_with_missing, axis=1)

In [22]:
X_full = all_data[:ntrain]
X_test = all_data[ntrain:]

In [23]:
#X_full = X_full.fillna(X_full.agg(lambda x: pd.Series.mode(x)[0], axis=0))
#X_test = X_test.fillna(X_test.agg(lambda x: pd.Series.mode(x)[0], axis=0))

In [24]:
X_test.head()

,Id,MSSubClass,LotArea,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleCondition
1460,1461,20,11622,Pave,Reg,Lvl,Inside,Gtl,NAmes,Feedr,...,140,0,0,0,120,0,0,6,2010,Normal
1461,1462,20,14267,Pave,IR1,Lvl,Corner,Gtl,NAmes,Norm,...,393,36,0,0,0,0,12500,6,2010,Normal
1462,1463,60,13830,Pave,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,212,34,0,0,0,0,0,3,2010,Normal
1463,1464,60,9978,Pave,IR1,Lvl,Inside,Gtl,Gilbert,Norm,...,360,36,0,0,0,0,0,6,2010,Normal
1464,1465,120,5005,Pave,IR1,HLS,Inside,Gtl,StoneBr,Norm,...,0,82,0,0,144,0,0,1,2010,Normal


In [25]:
X_full.columns[X_full.isnull().any()]
#X_full_Id = X_full.pop('Id')

Index([], dtype='object')

In [26]:
X_test.columns[X_test.isnull().any()]
#X_test_Id=X_test.pop('Id')

Index([], dtype='object')

In [27]:
X_full = X_full.select_dtypes(exclude=['object'])
X_test = X_test.select_dtypes(exclude=['object']) 


In [28]:
print(score_dataset(X_full, y_train))

19586.28366438356


In [29]:
#X_full = X_full_Id + X_full
#X_test = X_test_Id + X_test
X_test.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,...,GarageYrBlt,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
1460,1461,20,11622,5,6,1961,1961,896,0,0,...,1961.0,140,0,0,0,120,0,0,6,2010
1461,1462,20,14267,6,6,1958,1958,1329,0,0,...,1958.0,393,36,0,0,0,0,12500,6,2010
1462,1463,60,13830,5,5,1997,1997,928,701,0,...,1997.0,212,34,0,0,0,0,0,3,2010
1463,1464,60,9978,6,6,1998,1998,926,678,0,...,1998.0,360,36,0,0,0,0,0,6,2010
1464,1465,120,5005,8,5,1992,1992,1280,0,0,...,1992.0,0,82,0,0,144,0,0,1,2010


In [30]:


                       
model = RandomForestRegressor(n_estimators=100, random_state=0)
#log_y = np.log1p(y_train)
model.fit(X_full, y_train)
#predictions = np.expm1(model.predict(X_test)) 
predictions = model.predict(X_test)

output = pd.DataFrame({'Id': X_test.Id, 'SalePrice': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [31]:
X_test.shape

(1459, 27)